In [1]:
import sys
print(sys.executable)

C:\Users\kazii\anaconda3\envs\hydrogeo310\python.exe


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
pip install richdem

  Using cached richdem-0.3.4.tar.gz (329 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build richdem
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [100 lines of output]
  C:\Users\kazii\AppData\Local\Temp\pip-build-env-6tnqj9he\overlay\Lib\site-packages\setuptools\dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
  !!
  
          ********************************************************************************
          Usage of dash-separated 'description-file' will not be supported in future
          versions. Please use the underscore name 'description_file' instead.
          (Affected: richdem).
  
          By 2026-Mar-03, you need to update your project and remove deprecated calls
          or your builds will no longer be supported.
  
          See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
          ********************************************************************************
  
  !!
    opt =

##### At first, let's load the district shape file as a GeoDataFrame

In [1]:
dis_shp = Path(r"C:\Users\kazii\OneDrive - TUM\Desktop\USGS_PyforHyd\project_1\month1_watershed_delineation_terrain_analysis\dem\germany_Districts_level_3\germany_Districts_level_3.shp")
result_path = Path(r"C:\Users\kazii\OneDrive - TUM\Desktop\USGS_PyforHyd\project_1\month1_watershed_delineation_terrain_analysis\results")

NameError: name 'Path' is not defined

In [ ]:
districts = gpd.read_file(dis_shp)
districts

##### Now, let's extract our area of interest "Esslingen", located at the Stuttgart district of Germany, as a different GeoDataFrame

In [ ]:
esslingen_df = districts[districts['shape3'] == 'Esslingen']
display(esslingen_df)

#visualize the study area
fig, ax = plt.subplots(figsize=(7, 7))
esslingen_df.plot(ax=ax, edgecolor='black')
ax.set_title('Study Area: Esslingen, Stuttguart')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_facecolor('lightgray')

#fig.savefig(result_path/'study_area_vector.jpg', dpi=300)

##### Now load the downloaded raster tiles, include them in a dictionary and have a look on them

In [ ]:
dem_path = Path(r"C:\Users\kazii\OneDrive - TUM\Desktop\USGS_PyforHyd\project_1\month1_watershed_delineation_terrain_analysis\dem\esslingen_srtm_bulk")

In [ ]:
tile1 = dem_path/'tile1.tif'
tile2 = dem_path/'tile2.tif'
tile3 = dem_path/'tile3.tif'
tile4 = dem_path/'tile4.tif'

input_rasters = {
    'tile1': tile1,
    'tile2': tile2,
    'tile3': tile3,
    'tile4': tile4
}

In [ ]:
from rasterio.plot import show

In [ ]:
meta = {}
src_to_mosaic = []
fig, axes = plt.subplots(2, 2, figsize=(16,16))
for i, (tile, file) in enumerate(input_rasters.items()):
    with rasterio.open(file) as src:

        data = src.read(1)
        vmax, vmin = data.max(), data.min()
        meta[tile] = src.meta
        src_to_mosaic.append(src)
        
        ax = axes.flat[i]
        show(src, ax=ax, vmax=vmax, vmin=vmin, cmap='terrain')
        ax.set_title(tile)
        ax.set_facecolor('lightgrey')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        plt.colorbar(ax.images[0], ax=ax, label='Elevation, m', shrink=0.8)
fig.suptitle('Raster Tiles Covering the Study Area', fontsize=20, y=0.90)
#fig.savefig(result_path/'all_tiles.jpg')


In [ ]:
meta['tile1']

##### As our study area is covered by 4 seperate raster tiles, at this point, let's merge the them and get a single DEM tile

In [ ]:
from rasterio.merge import merge

In [ ]:
#listiling the datasets to mosaic
src_to_mosaic = [rasterio.open(file) for (tile, file) in input_rasters.items()]

#merging
mosaic, out_transform = merge(src_to_mosaic)

#preparing metadata of the merged raster
out_meta = src_to_mosaic[0].meta.copy()
out_meta.update({
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'transform': out_transform,
    'driver': 'GTiff'
})

#writing the merged dem as a new raster file
outfile = dem_path/'merged_dem.tif'

with rasterio.open(outfile, 'w', **out_meta) as dest:
    dest.write(mosaic)

for src in src_to_mosaic:
    src.close()

In [ ]:
#visualize the merged raster
merged_dem = dem_path/'merged_dem.tif'

fig, ax = plt.subplots(figsize=(8, 8))
with rasterio.open(merged_dem) as src:
    data = src.read(1)
    vmax = data.max()
    vmin = data.min()
    merged_dem_meta = src.meta
    
    show(src, ax=ax, vmax=vmax, vmin=vmin, cmap='terrain')
    ax.set_title('Merged Raster tile')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.colorbar(ax.images[0], ax=ax, shrink=0.8, label='Elevation, m')
#fig.savefig(result_path/'merged_tiles.jpg')

##### Let's clip the merged raster tile to the extend of our study area using the study area vector

In [ ]:
#the raster and vector needs to be in the same crs
print(f"The raster and the vector have same crs: {merged_dem_meta['crs'] == esslingen_df.crs}")
print(f"Raster's crs: {merged_dem_meta['crs']}")
print(f"Vector's crs: {esslingen_df.crs}")

In [ ]:
from rasterio.mask import mask

In [ ]:
#clip the raster
with rasterio.open(merged_dem) as src:
    out_array, out_transform = mask(src, esslingen_df.geometry, crop=True)
    out_meta = src.meta.copy()

#update metadata for writing new clipped raster
out_meta.update({
    'height': out_array.shape[1],
    'width': out_array.shape[2],
    'transform': out_transform
})

#write/save the new clipped raster
clipped_dem = dem_path/'clipped_dem.tif'
with rasterio.open(clipped_dem, 'w', **out_meta) as dest:
    dest.write(out_array)

In [ ]:
#visualize the clipped raster
fig, ax = plt.subplots()
with rasterio.open(clipped_dem) as src:
    data = src.read(1)
    vmax = data.max()
    vmin = data.min()
    show(src, ax=ax, cmap='terrain')
    ax.set_title('DEM of Esslingen')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_aspect('equal')
    plt.colorbar(ax.images[0], ax=ax, shrink=0.3, label='Elevation, m')
#fig.savefig(result_path/'clipped_dem.jpg')

In [ ]:
pip install GDAL

In [ ]:
!pip install richdem

In [ ]:
import richdem as rd